<a href="https://colab.research.google.com/github/Hammadtahir7/Card-Fraud-Detection/blob/master/CCFD_1DCNN_BCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

Saving BCDD.csv to BCDD.csv


In [4]:
import io
import pandas as pd
df2 = pd.read_csv(io.BytesIO(uploaded['BCDD.csv']),header=None)
df2.head()

,0,1,2,3,4,5,6,7
0,1.0,35.0,1.0,11753.0,2.0,4.0,38.0,1.0
1,1.0,24.0,1.0,5835.0,7.0,2.0,68.0,1.0
2,1.0,10.0,1.0,9020.0,11.0,14.0,63.0,1.0
3,1.0,39.0,1.0,9425.0,7.0,0.0,73.0,1.0
4,1.0,17.0,1.0,15672.0,42.0,0.0,43.0,1.0


In [0]:
features =df2.drop(df2.columns[7],axis=1)
labels=df2[df2.columns[7]]

In [6]:
import numpy as np
unique_elements, counts_elements = np.unique(labels, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[0.e+00 1.e+00]
 [5.e+04 5.e+04]]


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
features = scaler.fit_transform(features)
print(features[0])

X_train, X_test, y_train, y_test=train_test_split(features,labels,test_size=0.2, random_state=42)

#X_train=X_train.astype('float32')
#X_test=X_test.astype('float32')
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

[-0.83985099  0.71924274 -0.18851982  0.79273848 -1.15437249 -0.06768473
  1.35873136]
(80000, 7)
(20000, 7)
(80000,)
(20000,)


In [8]:
X_train=X_train.reshape(80000,1,7)
X_test=X_test.reshape(20000,1,7)
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [9]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

Using TensorFlow backend.


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, Dropout

model = Sequential()
model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1,7)))
model.add(Conv1D(filters=64, kernel_size=1, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [11]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m,precision_m, recall_m])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
#train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10)




Train on 80000 samples, validate on 20000 samples
Epoch 1/10





80000/80000 [==============================] - 6s 71us/step - loss: 0.2748 - acc: 0.8846 - f1_m: 0.8841 - precision_m: 0.8671 - recall_m: 0.9091 - val_loss: 0.2633 - val_acc: 0.8883 - val_f1_m: 0.8880 - val_precision_m: 0.8689 - val_recall_m: 0.9151
Epoch 2/10
80000/80000 [==============================] - 5s 63us/step - loss: 0.2639 - acc: 0.8877 - f1_m: 0.8874 - precision_m: 0.8692 - recall_m: 0.9130 - val_loss: 0.2651 - val_acc: 0.8881 - val_f1_m: 0.8873 - val_precision_m: 0.8728 - val_recall_m: 0.9091
Epoch 3/10
80000/80000 [==============================] - 5s 63us/step - loss: 0.2625 - acc: 0.8888 - f1_m: 0.8881 - precision_m: 0.8703 - recall_m: 0.9132 - val_loss: 0.2693 - val_acc: 0.8889 - val_f1_m: 0.8877 - val_precision_m: 0.8749 - val_recall_m: 0.9079
Epoch 4/10
80000/80000 [==============================] - 5s 61us/step - loss: 0.2624 - acc: 0.8887 - f1_m: 0.8881 - precision_m: 0.8712 - recall_m: 0.9121 - v

In [13]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test, verbose=0)


print("loss: "+"{:.2%}".format(loss));
print("acc: "+"{:.2%}".format(accuracy));
print("f1: "+"{:.2%}".format(f1_score));
print("precision: "+"{:.2%}".format(precision));
print("recall: "+"{:.2%}".format(recall));

loss: 28.46%
acc: 88.63%
f1: 88.35%
precision: 88.29%
recall: 89.08%


In [0]:
y_pred=model.predict_classes(X_test)

In [15]:
from sklearn import metrics
print("Accuracy: "+"{:.2%}".format(metrics.accuracy_score(y_test, y_pred)));
print("Precision: "+"{:.2%}".format(metrics.precision_score(y_test, y_pred)));
print("Recall: "+"{:.2%}".format(metrics.recall_score(y_test, y_pred)));
print("F1: "+"{:.2%}".format(metrics.f1_score(y_test,y_pred)));
print(metrics.confusion_matrix(y_test,y_pred));

Accuracy: 88.63%
Precision: 88.33%
Recall: 89.11%
F1: 88.72%
[[8784 1181]
 [1093 8942]]


In [16]:
for i in range(len(y_test)):
  if(y_test[i]==1):
    print(i)
    print(y_test[i])
    print(y_pred[i])
    print("-----")

2
1.0
[1]
-----
10
1.0
[1]
-----
11
1.0
[1]
-----
12
1.0
[1]
-----
13
1.0
[1]
-----
14
1.0
[1]
-----
16
1.0
[1]
-----
17
1.0
[1]
-----
21
1.0
[1]
-----
23
1.0
[1]
-----
24
1.0
[1]
-----
25
1.0
[1]
-----
29
1.0
[1]
-----
32
1.0
[0]
-----
33
1.0
[0]
-----
34
1.0
[0]
-----
35
1.0
[1]
-----
37
1.0
[1]
-----
39
1.0
[0]
-----
40
1.0
[1]
-----
42
1.0
[1]
-----
43
1.0
[1]
-----
44
1.0
[1]
-----
48
1.0
[0]
-----
50
1.0
[1]
-----
51
1.0
[1]
-----
52
1.0
[1]
-----
55
1.0
[1]
-----
59
1.0
[0]
-----
62
1.0
[1]
-----
64
1.0
[1]
-----
67
1.0
[1]
-----
68
1.0
[1]
-----
70
1.0
[1]
-----
71
1.0
[1]
-----
73
1.0
[1]
-----
74
1.0
[1]
-----
75
1.0
[1]
-----
76
1.0
[1]
-----
79
1.0
[1]
-----
80
1.0
[1]
-----
82
1.0
[0]
-----
89
1.0
[1]
-----
90
1.0
[1]
-----
92
1.0
[1]
-----
93
1.0
[1]
-----
98
1.0
[0]
-----
105
1.0
[1]
-----
106
1.0
[1]
-----
107
1.0
[1]
-----
117
1.0
[1]
-----
118
1.0
[1]
-----
121
1.0
[1]
-----
125
1.0
[1]
-----
127
1.0
[1]
-----
128
1.0
[0]
-----
129
1.0
[0]
-----
130
1.0
[1]
-----
132
